In [1]:
from pathlib import Path
import unidecode
import pandas as pd

In [2]:
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 250

In [3]:
df = pd.read_csv("../data/kanunum-nlp-doc-analysis-dataset.csv")

In [4]:
df.shape

(4142, 16)

### Alanlar
- **id**                    : İçerik kimliği
- **kategori**              : İçerik Türü/Kategorisi
- **baslik**                : İçerik Başlığı
- **rega_no**               : İçeriğin yayınlandığı Resmi Gazete Sayısı
- **mukerrer_no**           : İçeriğin yayınlandığı Resmi Gazete Mükerrer Numarası (0 ise mükerrer olmadığını belirtir)
- **rega_tarihi**           : İçeriğin yayınlandığı Resmi Gazete Tarihi (`date:yyyy-mm-dd` formatında)
- **kurum**                 : İçeriği(Genelge, Cumhurbaşkanlığı(CB) Kararnamesi, Tüzük, Yönetmelik, Tebliğ) çıkaran kurum adı
- **mevzuat_no**            : İçeriğin ilgili numarası (Detaylar için ilgili kategoriye bakınız) 
- **belge_sayi**            : Genelge üzerindeki Sayı ifadesinin değeri (sadece işlenebilenler)
- **mevzuat_tarihi**        : İçeriğin ilgili Tarihi (`date:yyyy-mm-dd` formatında)
- **donem**                 : Komisyon Raporunun dönemini (örn: "27. Dönem")
- **sira_no**               : Komisyon Raporunun Sıra Sayısı (örn: "230", "42 ek 1" vb.)
- **madde_sayisi**          : İçeriğin(Kanun, KHK, CB Kararnamesi, Tüzük, Yönetmelik) bölümlerini (Madde sayısını ve işlenemeyen hükümleri/maddeleri, _madde metinlerinin_ içindeki "Madde <sayi> - ..." ifadeleri hariç )
- **data_text**             : İçerik metni (Büyük/Küçük Harf, Noktalama ve "\n" içermektedir)
- **url**                   : İçerik metninin esas alındığı websayfasının linki (kaynak linkler güncel olmayabilir)
- **kanunum_url**           : İçeriğin Kanunum linki

**İçerik Başlıkları**, Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Tebliğ, Yönetmelik ve Tebliğ için içeriğin adı olan başlık kullanılırken Komisyon Raporu, Genelge ve Özelgelerde tanımlayıcı olmak üzere işelenerek üretilmiş bir başlık tercih edilmiştir.

**Kurum Adı**, Kanunum veritabanına işlenmiş hali ile kullanılmaktadır. Örneğin; Üniversite Yönetmeliklerinde kurum adı ayrı ayrı üniversite adı kullanmak yerine tek bir ad ("Üniversite") olarak işlenmiştir. Kurum tespiti yapılırken Bakanlık isimlerinin değişmiş olabileceği ayrıca dikkate alınmalıdır.

## Kategoriler
 Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Genelge, Tebliğ, Yönetmelik, Komisyon Raporu, Tüzük ve Özelge olmak üzere toplam 10 kategori bulunmaktadır.

Tüm içerikler için `kategori`, `baslik`, `data_text`, `url`, `kanunum_url` alanları sunulmuş olup `url` alanı içerik metninin esas alındığı sayfa linkini belirtmektedir. Diğer dikkate alınacak alanlar aşağıdaki gibi olup detaylar alt başlıklarda verilmiştir.


| Kategori                                                               | İçerik Sayısı     | İlgili alanlar (Üst Veriler)                                                                                              |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, data_text, url                                                       |
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | kategori, baslik, donem, sira_no, data_text, url                                                                          |
| [Genelge](#kategori-genelge)                                           | 385               | kategori, baslik, kurum, mevzuat_no, belge_sayi, mevzuat_tarihi, data_text, url                                           |
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tüzük](#kategori-tüzük)                                               | 82                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, data_text, url                                                |
| [Özelge](#kategori-özelge)                                             | 750               | kategori, baslik, kurum, mevzuat_tarihi, data_text, url M

### BELGE SAYISI

In [5]:
genelge_df = df[df.kategori.isin(["Genelge"])].copy()

for idx in genelge_df.index:
    standardized_txt = unidecode.unidecode(genelge_df.loc[idx].data_text).lower()
    try:
        genelge_df.loc[idx, "belge_sayi_pred"] = standardized_txt.replace(":", " ")\
                                                                .split('sayi ')[1]\
                                                                .split("\n")[0]
    except Exception as e:
        print(idx, ">", e)

920 > list index out of range
921 > list index out of range
1075 > list index out of range
1081 > list index out of range
1082 > list index out of range
1083 > list index out of range
1084 > list index out of range
1085 > list index out of range
1088 > list index out of range
1089 > list index out of range
1090 > list index out of range
1091 > list index out of range
1092 > list index out of range
1093 > list index out of range
1094 > list index out of range
1095 > list index out of range
1096 > list index out of range
1097 > list index out of range
1098 > list index out of range
1099 > list index out of range
1107 > list index out of range
1139 > list index out of range
1241 > list index out of range
1243 > list index out of range
1244 > list index out of range
1245 > list index out of range
1246 > list index out of range
1247 > list index out of range
1248 > list index out of range
1249 > list index out of range
1250 > list index out of range
1251 > list index out of range
1252 > lis

In [6]:
genelge_df[["belge_sayi_pred", "data_text"]][:5]

,belge_sayi_pred,data_text
917,b.07.0.gel.0.44/4412-17-047504,GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/47504\n\nTarih 15/12/1998\n\nSayı B.07.0.GEL.0.44/4412-17-047504\n\nKapsam \n\nT.C\n\nMALIYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.44/4412-17-047504 \n\nKONU:GELİR VERGİSİ KANU...
918,b.07.0.gel.0.71/7100-89 - 009461,VERGİ BARIŞI KANUNU UYGULAMA İÇ GENELGESİ SERİ NO: 2003/2\n\nTarih 08/03/2003\n\nSayı B.07.0.GEL.0.71/7100-89 - 009461\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.71/7100-89\n\nTARİH : 08.03.2003\n\nU...
919,b. 13.2.sgk.0.71.09.00/53>~? 3 0 mayis 2oi2,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nİnsan Kaynaklan Daire Başkanlığı\n\nSayı : B. 13.2.SGK.0.71.09.00/53>~? 3 0 MaYIS 2ÖÎ2\nKonu : Genelge Değişikliği\n\nGENELGE\n2012/20\n\n07/01/2010 tarihli ve 2010/3 sayılı “ Mal Bildirimi (e-mal bildi...
920,NaN,GENELGE\n\n\n2018/1 \n\nKredilerin Sınıflandırılması ve Bunlar İçin Ayrılacak Karşılıklara İlişkin Usul ve Esaslar Hakkında Yönetmeliğin (Yönetmelik) 7 inci maddesinin birinci fıkrasında; \n\n“Canlı veya donuk alacaklar için uygulanabilen yeniden...
921,NaN,Resmî Gazete Tarihi: 03.12.2021 Resmî Gazete Sayısı: 31678\nKonu: Engelli Haklan İzleme ve Değerlendirme Kurulu \n\nGENELGE \n\n2021/23 \n\n19/7/2013 tarihli ve 28712 sayılı Resmî Gazete’de yayımlanan 2013/8 sayılı Genelge ile kurulan “Engelli Ha...


In [7]:
genelge_df[~genelge_df.belge_sayi.isna()][["belge_sayi", "belge_sayi_pred"]]

,belge_sayi,belge_sayi_pred
1025,B.07.0.GEL.0.44/4412-1-20248,b.07.0.gel.0.44/4412-1-20248
1026,B.07.1.GİB.0.06.55-130[5511- 2801]-63856,b.07.1.gib.0.06.55-130[5511- 2801]-63856
1027,B.07.0.GEL.0.71/7103-275 /12360,b.07.0.gel.0.71/7103-275 /12360
1028,B.07.0.GEL.0.55-5511-2125/3599,b.07.0.gel.0.55-5511-2125/3599
1029,B.07.0.GEL.0.38/3802- 57/037774,b.07.0.gel.0.38/3802- 57/037774
1030,B.07.0.GEL.0.28/2810-13-866/02159,b.07.0.gel.0.28/2810-13-866/02159
1031,B.07.0.GEL.0.38/3802-43 / 001089,b.07.0.gel.0.38/3802-43 / 001089
1032,B.07.0.GEL.0.28/2810-13-866-068334,b.07.0.gel.0.28/2810-13-866-068334
1033,B.07.0.GEL.0.71/7103-140,b.07.0.gel.0.71/7103-140
1034,VUK.2/24491-212/07828,vuk.2/24491-212/07828


### MADDE SAYISI

In [8]:
madde_sayisi_cols = ["Kanun", "Kanun Hükmünde Kararname", "Cumhurbaşkanlığı Kararnamesi", "Tüzük", "Yönetmelik"]
madde_sayisi_df = df[df.kategori.isin(madde_sayisi_cols)].copy()

In [9]:
for idx in madde_sayisi_df.index:
    standardized_txt = unidecode.unidecode(madde_sayisi_df.loc[idx].data_text).lower()
    try:
        madde_sayisi_df.loc[idx, "madde_sayi_pred"] = standardized_txt.replace("-", ":")\
                                                                .split('\nmadde ')[-1]\
                                                                .split(":")[0]
    except Exception as e:
        print(idx, ">", e)

In [10]:
madde_sayisi_df[~madde_sayisi_df.madde_sayisi.isna()][["madde_sayisi", "madde_sayi_pred"]][:5]

,madde_sayisi,madde_sayi_pred
0,19,19
1,56,56
2,149,146
3,11,11
4,21,21


### MEVZUAT NO

In [11]:
mevzuat_no_cols = ["Kanun", "Kanun Hükmünde Kararname", "Genelge", "Cumhurbaşkanlığı Kararnamesi", "Tüzük", "Yönetmelik"]
mevzuat_no_df = df[df.kategori.isin(mevzuat_no_cols)]

In [12]:
mevzuat_no_df[700:]

,id,kategori,baslik,rega_no,mukerrer_no,rega_tarihi,kurum,mevzuat_no,belge_sayi,mevzuat_tarihi,donem,sira_no,madde_sayisi,data_text,url,kanunum_url
1012,2002,Genelge,"GELİR İDARESİ BAŞKANLIĞI, TAHSİLAT İÇ GENELGESİ SERİ NO: 2002/1",NaN,0.0,NaN,T.C. Gelir İdaresi Başkanlığı,2002/1,NaN,2002-04-02,NaN,NaN,0,TAHSİLAT İÇ GENELGESİ SERİ NO: 2002/1\n\nTarih 02/04/2002\nSayı B.07.0.GEL.0.36/3676-269/04713\n\nKapsam \nT.CMALİYE BAKANLIĞI\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.36/3676-269\nKONU : TAHSİLAT İÇ GENELGESİ SERİ NO : 2002/1\n\n...........,http://www.gib.gov.tr/node/89367,https://www.kanunum.com/content/348135
1013,2004,Genelge,"GELİR İDARESİ BAŞKANLIĞI, VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2010/2",NaN,0.0,NaN,T.C. Gelir İdaresi Başkanlığı,2010/2,NaN,2010-03-30,NaN,NaN,0,VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2010/2\n\nTarih 30/03/2010\n\nSayı B.07.1.GİB.0.05.70/7050-1/27939\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelir İdaresi Başkanlığı\n\nSayı : B.07.1.GİB.0.05.70/7050-1/27939 \n\nVERGİ KİMLİK NUMARASI ...,http://www.gib.gov.tr/node/89414,https://www.kanunum.com/content/348139
1014,2005,Genelge,"GELİR İDARESİ BAŞKANLIĞI, TAHSİLAT İÇ GENELGESİ SERİ NO: 2001/3",NaN,0.0,NaN,T.C. Gelir İdaresi Başkanlığı,2001/3,NaN,2001-05-31,NaN,NaN,0,TAHSİLAT İÇ GENELGESİ SERİ NO: 2001/3\n\nTarih 31/05/2001\nSayı B.07.0.GEL.0.37/3776-\n\nKapsam \nT.C.MALİYE BAKANLIĞI\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.37/3776-\nKONU : TAHSİLAT İÇ GENELGESİ SERİ NO: 2001/3\n\n.......................,http://www.gib.gov.tr/node/89322,https://www.kanunum.com/content/348141
1015,2007,Genelge,"GELİR İDARESİ BAŞKANLIĞI, VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2007/ 1",NaN,0.0,NaN,T.C. Gelir İdaresi Başkanlığı,2007/ 1,NaN,2007-07-06,NaN,NaN,0,VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2007/ 1\n\nTarih 06/07/2007\n\nSayı B.07.1.GİB.0.05.70/7033-158-60338\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelir İdaresi BaşkanlığI\n\nSayı: B.07.1.GİB.0.05.70/7033-158-60338\n\nVERGİ KİMLİK NUMARA...,http://www.gib.gov.tr/node/89380,https://www.kanunum.com/content/348145
1016,2008,Genelge,"GELİR İDARESİ BAŞKANLIĞI, VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2006/ 1",NaN,0.0,NaN,T.C. Gelir İdaresi Başkanlığı,2006/ 1,NaN,2006-10-31,NaN,NaN,0,VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2006/ 1\n\nTarih 31/10/2006\n\nSayı B.07.1.GİB.0.05.70/7033-158-84223\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelir İdaresi Başkanlığı\n\nSayı : B.07.1.GİB.0.05.70/7033-158-84223\n\nVERGİ KİMLİK NUMAR...,http://www.gib.gov.tr/node/89369,https://www.kanunum.com/content/348147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,5182,Yönetmelik,DENİZ TİCARET FİLOSUNUN GELİŞTİRİLMESİ VE GEMİ İNŞA TESİSLERİNİN TEŞVİKİ HAKKINDAKİ KANUNUN UYGULANMASINA İLİŞKİN YÖNETMELİK,25593,0.0,2004-09-24,"Ulaştırma, Denizcilik ve Haberleşme Bakanlığı",NaN,NaN,NaN,NaN,NaN,13,"Resmî Gazete Tarihi: 24.09.2004 Resmî Gazete Sayısı: 25593\nDENİZ TİCARET FİLOSUNUN GELİŞTİRİLMESİ VE GEMİ İNŞA TESİSLERİNİN TEŞVİKİ HAKKINDAKİ KANUNUN UYGULANMASINA İLİŞKİN YÖNETMELİK\n\nAmaç\nMadde 1 – Bu Yönetmeliğin amacı, 14/1/1982 tarihli v...",http://www.mevzuat.gov.tr/Metin.Aspx?MevzuatKod=7.5.5662&sourceXmlSearch=&MevzuatIliski=0,https://www.kanunum.com/content/131623
2531,5183,Yönetmelik,TEHLİKELİ MADDELERİN KARAYOLUYLA TAŞINMASI HAKKINDA YÖNETMELİK,31870,0.0,2022-06-18,Ulaştırma ve Altyapı Bakanlığı,NaN,NaN,NaN,NaN,NaN,48,Resmî Gazete Tarihi: 18.06.2022 Resmî Gazete Sayısı: 31870\nTEHLİKELİ MADDELERİN KARAYOLUYLA\n\nTAŞINMASI HAKKINDA YÖNETMELİK\n\nBİRİNCİ BÖLÜM\nBaşlangıç Hükümleri\nAmaç\nMadde 1 – (1) Bu Yönetmeliğin amacı; kamuya açık karayoluyla yapılacak tehl...,https://www.mevzuat.gov.tr/mevzuat?MevzuatNo=39586&MevzuatTur=7&MevzuatTertip=5,https://www.kanunum.com/content/21981818
2532,5184,Yönetmelik,MİLLÎ SAVUNMA UZMANLIĞI YÖNETMELİĞİ,28778,0.0,2013-09-27,Milli Savunma Bakanlığı,NaN,NaN,NaN,NaN,NaN,44,"Resmî Gazete Tarihi: 27.09.2013 Resmî Gazete Sayısı: 28778\nMİLLÎ 

In [13]:
print(madde_sayisi_df.iloc[0])

id                                                                                                                                                                                                                                                                         3
kategori                                                                                                                                                                                                                                                               Kanun
baslik                                                                                                                                                     BÜTÇE KANUNLARINDA YER ALAN BAZI HÜKÜMLERİN İLGİLİ KANUN VE  KANUN HÜKMÜNDE KARARNAMELERE EKLENMESİNE  DAİR KANUN
rega_no                                                                                                                                                                                          

In [14]:
print(mevzuat_no_df.iloc[700].data_text)

TAHSİLAT İÇ GENELGESİ SERİ NO: 2002/1

Tarih 02/04/2002
Sayı B.07.0.GEL.0.36/3676-269/04713

Kapsam 
T.CMALİYE BAKANLIĞI
Gelirler Genel Müdürlüğü

SAYI : B.07.0.GEL.0.36/3676-269
KONU : TAHSİLAT İÇ GENELGESİ SERİ NO : 2002/1

.................. VALİLİĞİ
(Defterdarlık: Gelir Müdürlüğüne)

İLGİ : a)29/03/1993 gün ve 1993/6 Seri Nolu Tahsilat İç Genelgesi,
b)25/09/1997 gün ve 1997/6 Seri Nolu Tahsilat İç Genelgesi.

Bilindiği gibi, tahsilatın hızlandırılması hususunda yayımlanan ilgi (a)'da kayıtlı genelgede, borçlu mükelleflerin kamu kurum ve kuruluşlarından hak ve alacağı bulunup bulunmadığı hususunda alacaklı vergi dairesi ile saymanlıklar arasında işbirliğinin sağlanması için yapılacak işlemler belirtilmiş, saymanlıklarca hak sahiplerine ödeme yapılmadan önce hak sahibinin bağlı bulunduğu vergi dairesi ile irtibat kurulması gerektiği bildirilmiştir.

Aynı genelgede, saymanlıklar ile vergi daireleri arasında kurulan bu sistemin Genel ve Katma Bütçeli İdareler Saymanlıklarını kapsadığı 